In [ ]:
import torch
import numpy as np
import os
import imageio
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from dataset import get_rays
    

In [ ]:
datapath = 'fox'
get_rays(datapath,mode='train')

